In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 
import torch
torch.set_num_threads(1)
import torch.multiprocessing as mp
import csv
from train import test_network, load_from_checkpoint
from utils import input_to_tensor_scalar

from resnet2 import ResNet2Heads
import logging

# import resource
# resource.setrlimit(
#     resource.RLIMIT_NOFILE,
#     (200000, 200000))

/Users/marshingjay/miniconda3/envs/python3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = 'resnet16_eps0.1_ep35.pt'
MODEL_NAME = 'resnet16_eps0.1'

NUM_PROCS = 4
ITERS = [1, 10, 50, 100, 500]
GAMES_PER_CONFIG = 200


In [3]:
csv_filename = ''

def write_results_to_csv(results):
    global csv_filename
    _, moves, high_square, score = results
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow([moves, high_square, score])

In [ ]:
logging.basicConfig(filename=f'{MODEL_NAME}_benchmark.log', filemode='a', level=logging.INFO, format='%(asctime)s %(message)s')
logging.info('Starting benchmark')

model, _, hypers, _, _, _ = load_from_checkpoint(MODEL_PATH, ResNet2Heads, load_replay_memory=False)
for iters in ITERS:
    logging.info(f'Benchmarking {iters} iterations...')
    with mp.Pool(NUM_PROCS) as p:
        csv_filename = f'{MODEL_NAME}_benchmark_{iters}.csv'
        results = []
        hypers.mcts_iters_eval = iters
        for _ in range(GAMES_PER_CONFIG):
            results.append(p.apply_async(test_network, (model, hypers, input_to_tensor_scalar), callback=write_results_to_csv, error_callback=print))
        for r in results:
            r.wait()
